<a href="https://colab.research.google.com/github/quick-wq/ConvertPythonCodeToOneLineTgBot/blob/patch-1/OCR_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# Optical Character Recognition using Tesseract

# By Suyesha Bhattacharjee


# IoT and Computer Vision Intern

Optical Character Recognition(OCR) has been a popular task in Computer Vision. Tesseract is the most open-source software available for OCR. It was initially developed by HP as a tool in C++. 
Since 2006 it is developed by Google. The original software is available as a command-line tool for windows. We are living in a python world. Because of its popularity,the tool is also available in python--developed and maintained as an opensource project.

### Step1. Install Pytesseract and tesseract-OCR in Google Colab.

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract tqdm numba
!wget -O /usr/share/tesseract-ocr/4.00/tessdata/rus.traineddata https://github.com/tesseract-ocr/tessdata_best/raw/main/rus.traineddata

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (7,336 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

### Step2. import libraries

In [ ]:
import pytesseract
import shutil
import os
import random
from tqdm import tqdm
import numba
from numba import jit
try:
    from PIL import Image
except ImportError:
    import Image

### Step3. Upload Image to the Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

path = "/content/drive/MyDrive/pictures" # Имя папки, укажите свое
if not os.path.exists(path):
    os.makedirs(path)

os.chdir(path)

Mounted at /content/drive


In [ ]:
#@title
import ipywidgets as widgets

# Создаем виджет чекбокс
checkbox = widgets.Checkbox(
    value=False,
    description='Переименовать изображения в img1.jpg; img2.jpg',
    indent=False
)

def rename_images(checkbox):
    if checkbox.value == True:
        file_list = [f for f in os.listdir(path) if f.endswith('.jpg')]

        for i, file_name in enumerate(file_list):
            new_file_name = 'img'+str(i+1)+'.jpg'
            os.rename(os.path.join(path, file_name), os.path.join(path, new_file_name))

        print("Все изображения успешно пронумерованы и переименованы!")

# Добавляем обработчик событий на изменение значения чекбокса
checkbox.observe(rename_images, 'value')

# Отображаем чекбокс
display(checkbox)

Checkbox(value=False, description='Переименовать изображения в img1.jpg; img2.jpg', indent=False)

In [13]:
@jit()
def process_image(file):
    extractedInformation = pytesseract.image_to_string(Image.open(file), lang='rus')
    # открываем файл для записи
    f = open(file.replace('.jpg', '.txt'), 'w', encoding='utf-8')
    try:
        f.write(extractedInformation)
    finally:
        # закрываем файл
        f.close()

file_list = [f for f in os.listdir(path) if f.endswith('.jpg')]

for file in tqdm(file_list):
    process_image(file)

  0%|          | 0/23 [00:00<?, ?it/s]<ipython-input-13-02e668d2a3dc>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "process_image" failed type inference due to: Untyped global name 'open': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-13-02e668d2a3dc>", line 5:
def process_image(file):
    <source elided>
    # открываем файл для записи
    f = open(file.replace('.jpg', '.txt'), 'w', encoding='utf-8')
    ^

  @jit()
  0%|          | 0/23 [00:00<?, ?it/s]


NumbaNotImplementedError: ignored

In [ ]:
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import multiprocessing

resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

def process_image(file):
    extractedInformation = pytesseract.image_to_string(Image.open(file), lang='rus+eng')
    with open(file.replace('.jpg', '.txt'), 'w', encoding='utf-8') as f:
        f.write(extractedInformation)

if __name__ == '__main__':
    file_list = [f for f in os.listdir(path) if f.endswith('.jpg')]

    with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        futures = [executor.submit(process_image, file) for file in file_list]
        max_ = len(futures)
        with tqdm(total=max_) as pbar:
            for i, _ in tqdm(enumerate(concurrent.futures.as_completed(futures))):            
                pbar.update()

100%|██████████| 23/23 [03:34<00:00,  8.81s/it]
23it [03:34,  9.35s/it]
100%|██████████| 23/23 [03:34<00:00,  9.35s/it]


## Step4. Text Extraction

### Step5. Printing the extracted information

In [ ]:
file_list = [f for f in os.listdir(path) if f.endswith('.jpg')]

for file in tqdm(file_list):
    extractedInformation = pytesseract.image_to_string(Image.open(file), lang='rus')
    with open(file.replace('.jpg', '.txt'), 'w', encoding='u8') as f:
        f.write(extractedInformation)

 96%|█████████▌| 22/23 [02:15<00:06,  6.57s/it]

In [ ]:
import multiprocessing
print("Количество ядер:", multiprocessing.cpu_count())

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("Количество CUDA-ядер:", gpu.attributes['num_cores'])

Количество ядер: 2
